In [1]:
import os
import argparse
import cv2
import sys
import numpy as np
import insightface
from tqdm import tqdm
from insightface.app import FaceAnalysis 
import pickle
import time
from leech import Leech
import sklearn
import math
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


app = FaceAnalysis(providers=['CPUExecutionProvider'])

app.prepare(ctx_id=0)

#feret
#def get_name(s):
#    return s[:5] 

#lfw
def get_name(s):
    return s[:-9] 


def get_names_vecs(folder):

    cache_file = 'embedding_'+folder
    if os.path.exists(cache_file):
        print('load from cache')
        data = pickle.load(open(cache_file,'rb'))
        names, vecs = data
        return names, vecs 
    
    names = []
    vecs = []
    name_list=[]
    cnt=0
    for f in os.listdir(folder):
        if f[-3:]=='jpg':
            name_list.append(f)

    for name in tqdm(name_list):
        img = cv2.imread(folder +'/'+ name)
        #print(folder+name)
        faces = app.get(img)
        if len(faces)!=1:
            continue
        
        img_embedding = faces[0].normed_embedding
        names.append(get_name(name))
        vecs.append(img_embedding)
    
    data = (names, vecs)
    pickle.dump(data,open(cache_file,'wb'))
    
    return names,vecs

names, vecs = get_names_vecs('lfw')

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kzoacn/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kzoacn/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kzoacn/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kzoacn/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kzoacn/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [2]:
import random
import numpy as np
random.seed(12345)

In [3]:
q = random.choice(range(len(names)))
print(q,names[q])

6825 Roger_Federer


In [4]:
alpha = 2**20

def round(v):
    return [int(x*alpha) for x in v]

def padding(v):
    v=list(v.copy())
    while len(v)%24!=0:
        v.append(0)
    return np.array(v)
    
def decode_h(v,t):

    
    v=padding(v)
    
    v=round(v)

    ds = []
    os = []
    for i in range(len(v)//24):
        sub_v = v[i*24:i*24+24]
        decoded = Leech.decode(sub_v,t*alpha)
        offset = np.array(sub_v) - np.array(decoded)

        ds.append(decoded)
        os.append(offset)
    return np.concatenate(ds), np.concatenate(os)
    
    

In [5]:
point, offset = decode_h(vecs[q],0.08)

In [6]:
from multiprocessing import Pool
ground_truth = False
for i in tqdm(range(len(names))):
    if i!=q and names[i]==names[q]:
        ground_truth = True

print('ground_truth',ground_truth)

def work(vec):
    p,_ = decode_h(padding(vec) - offset/alpha ,0.08)
    return p
    

with Pool(16) as p:
    r = list(tqdm(p.imap(work, vecs), total=len(vecs)))

100%|████████████████████████████████| 10616/10616 [00:00<00:00, 5644154.05it/s]

ground_truth True



100%|█████████████████████████████████████| 10616/10616 [46:30<00:00,  3.80it/s]


In [7]:
import hashlib


In [10]:
psi_query = hashlib.sha256(point).hexdigest()

In [12]:
psi_data=[]
for i in range(len(names)):
    if i==q:
        continue
    h = hashlib.sha256(r[i]).hexdigest()
    psi_data.append(h)

In [13]:
with open('psi_query.csv','w') as f:
    f.write(psi_query)

with open('psi_data.csv','w') as f:
    for h in psi_data:
        f.write(h+'\n')


In [14]:
import subprocess


In [15]:
subprocess.run('bash run_psi.sh'.split())

CompletedProcess(args=['bash', 'run_psi.sh'], returncode=0)

reading set... reading set... 0ms
connecting as server at address localhost:12122ms
connecting as client at address localhost:1212 2ms
Validating set sizes...  0ms
Validating set sizes... 0ms
running PSI... 0ms
running PSI... 32ms
Done
32ms
Writing output to ./psi_query.csv.out 0ms
intesection_size = 15
